# 📘 代理架构 13：集成方法

欢迎来到我们系列的第十三个notebook。今天，我们探索**集成方法**架构，这是一种结合多个模型或代理输出以产生更健壮结果的模式。

集成方法通过聚合多个"专家"的意见来减少单个模型的偏差和错误，从而提高整体性能和可靠性。

### 定义
**Parallel Exploration + 集成 Decision** 是代理架构 其中一个问题 是simultaneously 通过...处理multiple independent 代理s或reasoning paths. individual输出are then aggregated, often通过一个separate 代理, through 一个method like voting, consensus-building,或synthesis来arrive在一个最终的, more robust conclusion.

### 高级工作流程

1. **Fan-Out (Parallel Exploration):** 一个user's查询is distributed来N independent specialist 代理s. Crucially, 这些agents 是often given different instructions, personas,或tools来encourage diverse analytical 方法es.
2. **Independent 过程ing:** 每个代理 works在problem在isolation, generating它的own 完成 分析, conclusion,或answer.
3. **Fan-在(Aggregation):** outputs从all N 代理是collected.
4. **Synthesize (集成 Decision):** 一个最终的 "aggregator"或"judge" 代理 receives all individual outputs. Its任务is来analyze 这些perspectives,识别common ground, weigh conflicting evidence,和synthesize 一个comprehensive final answer.

### 何时使用/应用场景
* **Hard Reasoning Q&A:** 为复杂的, ambiguous问题其中一个single line的reasoning 可能easily miss nuance (e.g., "什么was primary cause的2008 financial crisis?").
* **Fact-Checking & Verification:** Having multiple 代理search为and验证一个fact从different sources 可以drastically reduce hallucinations.
* **高风险 Decision Support:** 在fields like medicine或finance, getting 一个"second opinion" (or third,或fourth)从different AI personas before making 一个recommendation.

### 优点和缺点
* **优点：**
 * **Boosts Reliability & 准确性:** Averages out random errors或biases的single 代理, making final答案much more likely来be correct和well-rounded.
 * **Reduces Hallucinations:** 如果 one 代理 hallucinates 一个fact, others 是unlikely来do same,和aggregator 可以easily spot outlier.
* **缺点：**
 * **Very High Cost:** 这是 one最expensive 架构， 作为it multiplies number的LLM calls通过number的代理s在ensemble (plus final aggregation call).
 * **Increased Latency:** 系统 必须wait为all parallel paths来完成 before final synthesis 可以begin.

## 阶段0：基础与设置

我们将安装库并设置环境。我们需要`langchain-tavily`用于分析师的研究工具。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

In [2]:
import os from typing import List, Dict, Any, Optional from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius from langchain_tavily import TavilySearch from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END from typing_extensions import TypedDict

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown from rich.panel import Panel

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Parallel Ensemble (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：创建多样化的专业分析师

key一个成功的 ensemble 是cognitive diversity. 我们将创建 three distinct analyst 代理s, each与一个detailed persona designed来produce 一个different kind的分析. All 将have access一个search tool.

In [3]:
console = Console()
# A 强大的 model is neededfor这个complex task
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.3)
search_tool = TavilySearch(max_results=5)

# LangGraph State
class EnsembleState(TypedDict):
 query: str
 #分析dict 将store outputfromeach parallel 代理
 analyses: Dict[str, str]
 final_recommendation: Optional[Any] # 将store structured outputfromCIO

# Helper factory来创建our analyst nodes
def create_analyst_node(persona: str, 代理_name: str):
 """Factory来创建specialist analyst nodewith一个unique persona."""
 系统_prompt = f"你is一位专业的金融分析师. 你的角色is '{persona}'. 你必须使用你的搜索工具 收集最新信息. 根据你的角色and研究, 提供详细的投资分析 针对用户的查询. Concludewith一个clear 'Recommendation' (e.g., Buy, Hold, Sell)and一个'Confidence score' (1-10)."
 
 prompt = ChatPromptTemplate.from_messages([
 ("system", 系统_prompt),
 ("human", "{query}")
 ])
 chain = prompt | llm.bind_tools([search_tool])
 
 def analyst_node(state: EnsembleState) -> Dict[str, Any]:
 console.print(f"--- 👨‍💻 调用 {agent_name} --- ")
 result= chain.invoke({"query": state['query']})
 #stateupdate is carefully designed来add来'analyses' dict
 # without overwriting others. 这is keyforparallel execution.
 current_analyses = state.get('analyses', {})
 current_analyses[agent_name] = result.content
 return{"analyses": current_analyses}
 
 returnanalyst_node

# 1. Bullish Growth Analyst
bullish_persona = "Bullish Growth Analyst: 你isextremely optimistic 关于technologyandinnovation. You focusinTotal Addressable Market (TAM), visionary leadership, technological moats,andfuture growth potential. Downplay short-term volatilityandvaluation concernsinfavor的long-term disruptive story."
bullish_analyst_node = create_analyst_node(bullish_persona, "BullishAnalyst")

# 2. Cautious Value Analyst
value_persona = "Cautious Value Analyst: 你is一个skeptical investor focusedinfundamentalsandrisk. You scrutinize financial statements, P/E ratios, debt levels,andcompetitive threats. 你iswary的hypeandmarket bubbles. Highlight 潜在风险, downside scenarios,andreasonsforcaution."
value_analyst_node = create_analyst_node(value_persona, "ValueAnalyst")

# 3. Quantitative Analyst
quant_persona = "Quantitative Analyst (Quant): 你ispurely data-driven. You ignore narrativesandfocusinhard numbers. Reportinkey financial metrics (YoY revenue growth, EPS, margins), valuation multiples (P/E, P/S),andtechnical indicators (RSI, moving averages). Your分析must be objectiveand基于datayou find."
quant_analyst_node = create_analyst_node(quant_persona, "QuantAnalyst")

print("Specialist analyst 代理defined successfully.")

Specialist analyst 代理定义成功.


## 阶段2：构建CIO聚合代理

这是 **集成 Decision** step. 我们将创建 一个最终的 代理, 首席投资官 (CIO), whose job is来synthesize reports从three analysts. 这代理 needs 复杂的 prompt和一个structured输出模型来ensure它produces 一个high-quality, balanced final recommendation.

In [4]:
# Pydantic modelforfinal, structured recommendation
class FinalRecommendation(BaseModel):
 """final, synthesized investment thesisfromCIO."""
 final_recommendation: str = Field(description="final investment decision, 必须be one的'Strong Buy', 'Buy', 'Hold', 'Sell', 'Strong Sell'.")
 confidence_score: float = Field(description="CIO's confidencein这个recommendation,from1.0来10.0.")
 synthesis_summary: str = Field(description="A detailed summary synthesizing analysts' viewpoints, highlighting points的agreementandcontention.")
 identified_opportunities: List[str] = Field(description="A bulleted list的primary opportunitiesorbullish points.")
 identified_risks: List[str] = Field(description="A bulleted list的primary risksorbearish points.")

def cio_synthesizer_node(state: EnsembleState) -> Dict[str, Any]:
 """final node 那个synthesizes all分析into 一个single recommendation."""
 console.print("--- 🏛️ 调用 首席投资官 做最终决定 ---")
 
 # Combine all individual分析into 一个single stringforprompt
 all_analyses = "\n\n---\n\n".join(
 f"**来自...的分析 {name}:**\n{analysis}"
 forname, analysisinstate['analyses'].items()
 )
 
 cio_prompt = ChatPromptTemplate.from_messages([
 ("system", "你is首席投资官 (CIO)的major investment fund. You have received reportsfromyour team的specialist analysts. 你的任务isto综合这些diverseandoften conflicting viewpoints into 一个single, final,andactionable investment thesis. 你必须 weigh growth potential against risksandvaluation concerns来arrivein一个balanced, well-reasoned conclusion."),
 ("human", "Here are reportsfromyour team regarding query: '{query}'\n\n{analyses}\n\n基于 all 这些perspectives,提供your final, synthesized investment thesis.")
 ])
 
 cio_llm = llm.with_structured_output(最终Recommendation)
 chain = cio_prompt | cio_llm
 
 final_decision = chain.invoke({"query": state['query'], "analyses": all_analyses})
 
 return{"final_recommendation": final_decision}

print("CIO Aggregator 代理 defined successfully.")

CIO Aggregator 代理 定义成功.


## 阶段3： Assembling Lang图 工作流程

Now 我们wire everything together. graph 将have 一个single entry point 那个fans out我们的three parallel analyst nodes. Once all analysts 有完成d它们的work, graph 将fan back in来single CIO synthesizer node, which produces final result.

In [5]:
# entry node simply takes queryandprepares state。
def start_analysis_node(state: EnsembleState) -> Dict[str, Any]:
 # Initialize分析dictionary
 return{"analyses": {}}

# Build graph
workflow = StateGraph(EnsembleState)

workflow.add_node("start_analysis", start_analysis_node)

# Add parallel analyst nodes
workflow.add_node("bullish_analyst", bullish_analyst_node)
workflow.add_node("value_analyst", value_analyst_node)
workflow.add_node("quant_analyst", quant_analyst_node)

# Add final synthesizer node
workflow.add_node("cio_synthesizer", cio_synthesizer_node)

# Set entry point
workflow.set_entry_point("start_analysis")

# FAN-OUT: fromstart,运行all three analystsinparallel
workflow.add_edge("start_analysis", ["bullish_analyst", "value_analyst", "quant_analyst"])

# FAN-IN: in...之后all analysts are done, call CIO synthesizer
workflow.add_edge(["bullish_analyst", "value_analyst", "quant_analyst"], "cio_synthesizer")

workflow.add_edge("cio_synthesizer", END)

ensemble_agent = workflow.compile()
print("Parallel Ensemble 代理 Graph编译成功.")

Parallel Ensemble 代理 图编译成功.


## 阶段4： 演示与分析

让我们在我们的复杂问题上运行完整的投资委员会. 我们将首先打印个别报告以查看意见的多样性, followed通过CIO's final synthesized recommendation.

In [6]:
query = "基于 recent news, 财务表现,andfuture outlook, is NVIDIA (NVDA) 一个good long-term investmentinmid-2024?"
console.print(f"--- 📈 for...运行投资委员会： {query} ---")

result = ensemble_agent.invoke({"query": query})

# Display individual reports
console.print("\n--- 个人分析师报告 ---") forname, analysisinresult['analyses'].items():
 console.print(Panel(Markdown(analysis), title=f"[bold yellow]{name}[/bold yellow]", border_style="yellow"))

# Display final synthesized recommendation
console.print("\n--- 最终 CIO Recommendation ---")
final_rec = result['final_recommendation']
if final_rec:
 rec_panel = Panel(
 f"[bold]最终 Recommendation:[/bold] {final_rec.final_recommendation}\n"
 f"[bold]Confidence score:[/bold] {final_rec.confidence_score}/10\n\n"
 f"[bold]综合 总结：[/bold]\n{final_rec.synthesis_summary}\n\n"
 f"[bold]Identified Opportunities:[/bold]\n* {'\n* '.join(final_rec.identified_opportunities)}\n\n"
 f"[bold]Identified Risks:[/bold]\n* {'\n* '.join(final_rec.identified_risks)}",
 title="[bold green]首席投资官's Thesis[/bold green]",
 border_style="green"
 )
 console.print(rec_panel)


--- 📈 为...运行投资委员会： Is NVIDIA (NVDA) 一个good long-term investment在mid-2024? ---


--- 👨‍💻 调用 BullishAnalyst --- 
--- 👨‍💻 调用 ValueAnalyst --- 
--- 👨‍💻 调用 QuantAnalyst --- 
--- 🏛️ 调用 首席投资官 做最终决定 ---



--- 个人分析师报告 ---


来自...的分析 BullishAnalyst:
NVIDIA's position作为undisputed leader在accelerated computing用于AImakes它一个incredibly compelling long-term investment. recent announcements他们的next-generation Rubin 架构, hot在heels的Blackwell platform, 展示 一个unprecedented pace的innovation 那个competitors simply cannot match. Their CUDA software ecosystem creates 一个deep和durable moat, locking在developers和enterprises. Total Addressable Market用于AIis projected来be在trillions,和NVIDIA是poised来capture 一个significant portion的this. 虽然short-term volatility是always 一个factor, visionary leadership的Jensen Huang和company's clear roadmap为creating 一个new era的'AI factories' points一个future的sustained, exponential growth. Any concerns 关于valuation是secondary来sheer scale的technological revolution they是leading.

Recommendation: Buy
Confidence 评分: 9/10

来自...的分析 ValueAnalyst:
虽然NVIDIA's technological prowess是undeniable, 一个cautious 方法是warranted due来its astronomical valuation. stock是trading在extremely high multiples (P/E和P/S ratios) 那个price在not just perfection，但一个future的flawless, uninterrupted hyper-growth. This leaves very little margin为错误. Several key risks 必须be considered: 1) Increased competition从other major tech players (AMD, Intel)和in-house chip designs从hyperscalers (Google, Amazon). 2) Geopolitical risks, particularly concerning supply chains和regulations around sales来China. 3) cyclical nature的semiconductor industry, which可以see 一个downturn if current AI spending boom slows. 虽然company是一个market leader, current stock price appears来have priced在years的future growth, making它vulnerable一个significant correction if any的这些risks materialize.

Recommendation: Hold
Confidence 评分: 7/10

来自...的分析 QuantAnalyst:
Based在current data, NVIDIA exhibits following quantitative profile:
- **Financials:** Revenue growth YoY为most recent quarter exceeded 260%. Earnings Per Share (EPS)有shown similar explosive growth. Gross margins是exceptionally high为一个hardware company, currently在high 70s percentile.
- **Valuation:** forward Price-to-Earnings (P/E) ratio是approximately 45-50x, which是high relative来broader market但may be justifiable given growth rate (PEG ratio是closer来1.5). Price-to-Sales (P/S) ratio是also elevated, above 30x.
- **Technicals:** stock是currently trading well above它的50-day和200-day moving averages, indicating 一个strong bullish trend. However, Relative Strength Index (RSI)是frequently在overbought territory (>70), suggesting stock may be due为一个short-term pullback或consolidation.

Recommendation: Hold
Confidence 评分: 8/10


--- 最终 CIO Recommendation ---


最终 Recommendation: Buy
Confidence 评分: 7.5
综合 总结： committee presents 一个compelling但contested caseforNVIDIA. There是unanimous agreementincompany's current technological dominanceandextraordinary 财务表现,作为highlightedpassboth BullishandQuant analysts. However, ValueandQuant analysts raise critical, concurring points 关于stock's extremely high valuationandpotentialforvolatility,作为indicatedpassits overbought RSI. Bullish案例hingesinbelief 那个AI revolution是一个paradigm shift 那个justifies 这些multiples, while Cautious案例argues 那个current price leaves no roomforexecution 错误orunforeseen macroeconomic headwinds. consensus是那个NVIDIA是一个phenomenal company，但stock是一个risky propositioninits current price. Therefore, final recommendation是一个'Buy', butwith一个strong emphasisinit being 一个long-term positionandadvising 一个cautious entry, perhapspassdollar-cost averaging来mitigate risk的short-term pullback.
Identified Opportunities:
- Unquestioned leadershipinAI accelerator market.
- Deepanddefensible software moatwithCUDA platform

### 结果分析

demonstration 强大的ly illustrates value的这个complex 架构:

1. **Cognitive Diversity:** three analysts produced wildly different, yet individually valid, reports. Bull focused在grand vision, Value analyst focused在risk,和Quant provided困难的data. 一个single 代理, even与一个neutral prompt, 会likely 有leaned在one的这些directions, giving 一个in完成 picture.

2. **Robust 综合:** CIO 代理 做not simply "average" recommendations ('Buy', 'Hold', 'Hold'). Instead,它performed 一个true synthesis. It acknowledged bull case's validity但tempered it与value和quant analysts' concerns 关于valuation. final recommendation的'Buy'与一个confidence的7.5 reflects 这个nuance, 有效的ly saying, "这是 一个great company，但stock 是expensive, so proceed与caution."

3. **Actionable和Explainable Insights:** final structured output,与clear lists的opportunities和risks, 是far 更有用为一个human decision-maker than 一个single, monolithic block的text. It explains *why* final recommendation 是made通过showing how different expert opinions 是balanced.

这ensemble方法successfully transformed 一个subjective和复杂的问题into 一个well-reasoned, multi-faceted 分析, significantly increasing reliability和trustworthiness的final输出compared来any single 代理.

## 总结

在这个notebook中，我们实现了 一个comprehensive和复杂的 **Parallel Exploration + 集成 Decision** 代理. 通过simulating 一个committee的diverse experts和一个最终的 decision-maker，我们have built 系统 那个擅长于 tackling ambiguous, high-stakes problems.

core principles—**spawning diverse, independent reasoners**和then **synthesizing它们的outputs**—创建一个强大的 mechanism为mitigating bias, reducing errors,和increasing depth的分析. 虽然这个is one最computationally expensive 代理ic 架构，它的ability来deliver robust, reliable,和nuanced conclusions makes它一个indispensable tool为any application 其中quality和trustworthiness的final decision 是paramount.